In [ ]:
import gradio as gr
from openai.types.responses import ResponseTextDeltaEvent
from pydantic import BaseModel, Field
from agents import Agent, trace, Runner, WebSearchTool
from dotenv import load_dotenv
from IPython.display import display, Markdown
import asyncio
from agents.model_settings import ModelSettings

load_dotenv(override=True)

class Evaluation(BaseModel):
    is_satisfactory: bool = Field(description="Whether the user's answers are satisfactory to our questions.")
    feedback: str = Field(description="Feedback on the user's answers.")
    score: int = Field(description="Score between 0 and 1 based on the quality of the answers.")

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")

class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")

HOW_MANY_SEARCHES = 3

planner_instructions= f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."
planner_agent = Agent(
    name="PlannerAgent",
    instructions=planner_instructions,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

writer_instructions = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)
writer_agent = Agent(
    name="WriterAgent",
    instructions=writer_instructions,
    model="gpt-4o-mini",
    output_type=ReportData,
)

search_instructions = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."
search_agent = Agent(
    name="Search agent",
    instructions=search_instructions,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

eval_instructions = """
You need to evaluate whether the user's answers are satisfactory to our questions. 
If not, you need to tell the user where it is lacking.
Do not be strict, if the answers are enough to continue with the research, you can say it is satisfactory.
"""
evaluator_agent = Agent(
    name="Evaluator",
    instructions=eval_instructions,
    model="gpt-4o-mini",
    output_type=Evaluation,
)
clarifying_agent = Agent(
    name="Clarifying Questions Agent",
    instructions="You are a helpful assistant that asks three clarifying questions to the user.",
    model="gpt-4o-mini",
)

async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def run_research(query: str, questions: str, answers: str):
    """ Run the full research pipeline: plan searches, perform searches, write report, and send email """
    with trace("Research trace"):
        print("Starting research...")
        input = f"Query: {query}\nQuestions: {questions}\nAnswers: {answers}"
        search_plan = await plan_searches(input)
        search_results = await perform_searches(search_plan)
        report = await write_report(query, search_results)
        print("Hooray!")
        return report.markdown_report

async def call_questions_stream(topic: str):
    """Streaming version - THIS IS THE FIX"""
    yield gr.update(visible=True, value="")
    with trace("Clarifying Questions"):
        full_text = ""
        result = Runner.run_streamed(clarifying_agent, input=topic)
        async for event in result.stream_events():
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                full_text+=event.data.delta
                yield full_text

async def evaluate_and_research(topic: str, questions: str, answers: str):
    """Evaluate answers first, then run research if satisfactory"""
    # Run evaluator
    input = f"Topic:\n{topic}\n\nQuestions:\n{questions}\n\nAnswers:\n{answers}\n\n\
        Criteria: Evaluate if the answers provide enough focus and specificity for conducting targeted research."
    evaluation = await Runner.run(evaluator_agent, input=input)
    evaluation_result = evaluation.final_output
    if not evaluation_result.is_satisfactory and evaluation_result.score<0.5:
        # Return feedback if not satisfactory
        feedback_message = f"❌ **Answers need improvement:**\n\n{evaluation_result.feedback}\n\nPlease revise your answers and click the button again."
        yield gr.update(visible=True, value=feedback_message)
        return  # Stop here if not satisfactory
    
    # If satisfactory, proceed with research
    success_message = "✅ **Answers are satisfactory! Starting research...**\n\n"
    yield gr.update(visible=True, value=success_message)
    
    # Run the actual research
    research_result = await run_research(topic, questions, answers)
    yield gr.update(visible=True, value=research_result)



In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# 🔬 Deep Research Assistant")
    gr.Markdown("Enter your research topic below. I'll ask clarifying questions to better understand your needs, then conduct comprehensive research.")
    
    with gr.Row():
        with gr.Column(scale=1):
            topic = gr.Textbox(
                label="Research Topic",
                placeholder="e.g., What is the best AI Agent framework in 2025?",
                lines=2
            )
            clarify_btn = gr.Button("🤔 Get Clarifying Questions", variant="primary", size="lg")
            
            questions = gr.TextArea(visible=False, label="Clarifying Questions")
            answers = gr.Textbox(
                visible=False,
                label="Your Answers",
                placeholder="Please answer the questions above to help refine the research scope...",
                lines=5
            )
            
            research_btn = gr.Button("🚀 Start Deep Research", visible=False, variant="primary", size="lg")
        
        with gr.Column(scale=1):
            report = gr.Markdown(visible=False, label="Research Report", header_links=True,
                    show_copy_button=True,line_breaks=True,sanitize_html=True, height=500,)

    clarify_btn.click(
        fn=call_questions_stream,
        inputs=topic,
        outputs=[questions]
    ).then(
        fn=lambda: (gr.update(visible=True, interactive=True), gr.update(visible=True)),
        inputs=None,
        outputs=[answers, research_btn]
    )

    
    research_btn.click(
        fn=evaluate_and_research,
        inputs=[topic, questions, answers],
        outputs=[report]
    )


demo.launch(inbrowser=True, inline=False)